In [ ]:
import tensorflow as tf
import os
import mne
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt




In [ ]:
df = pd.read_csv("eeg_labeled_dataset.csv") 


In [ ]:
df['Label'] = df['Label'].apply (lambda x:3  if x == 'Delta' else 2 if x == 'Theta' else 0 if x == 'Alpha' else 1 if x == 'Beta' else 0 )

In [ ]:


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

train_data, test_data  = train_test_split(df,  train_size=0.8, random_state=42, stratify=df['Label'])  
train_data.shape, test_data.shape



In [ ]:
X_train = train_data.drop(columns=['Label'])
y_train = train_data['Label']

X_test = test_data.drop(columns=['Label'])
y_test = test_data['Label']

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# X_train_np = X_train.to_numpy()
# y_train_np = y_train.to_numpy()

# X_test_np = X_test.to_numpy()
# y_test_np = y_test.to_numpy()




In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Convert labels to categorical one-hot encoding
y_train_cat = to_categorical(y_train, num_classes=4)
y_test_cat = to_categorical(y_test, num_classes=4)

# Define the 1D CNN model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))  # Input layer
model.add(Dense(32, activation='relu'))  # Hidden layer
model.add(Dense(5, activation='softmax')) # Output layer (5 classes)


# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use sparse_categorical_crossentropy for integer labels
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2) # Adjust epochs and batch_size


# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# Make predictions
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1) # Get the predicted class labels

#Further analysis and visualization of results can be added here.
